In [1]:
#!pip install splinter

In [2]:
# Import dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from selenium import webdriver

In [3]:
# Set up Splinter
browser = Browser('chrome')

The chromedriver version (116.0.5845.96) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with the detected chrome version (131.0.6778.70); currently, chromedriver 131.0.6778.69 is recommended for chrome 131.*, so it is advised to delete the driver in PATH and retry
The chromedriver version (116.0.5845.96) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with the detected chrome version (131.0.6778.70); currently, chromedriver 131.0.6778.69 is recommended for chrome 131.*, so it is advised to delete the driver in PATH and retry
The chromedriver version (116.0.5845.96) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with the detected chrome version (131.0.6778.70); currently, chromedriver 131.0.6778.69 is recommended for chrome 131.*, so it is advised to delete the driver in PATH and retry


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 116
Current browser version is 131.0.6778.70 with binary path /Applications/Google Chrome.app/Contents/MacOS/Google Chrome
Stacktrace:
0   chromedriver                        0x000000010300e65c chromedriver + 4318812
1   chromedriver                        0x0000000103006d00 chromedriver + 4287744
2   chromedriver                        0x0000000102c387ec chromedriver + 296940
3   chromedriver                        0x0000000102c66980 chromedriver + 485760
4   chromedriver                        0x0000000102c62514 chromedriver + 468244
5   chromedriver                        0x0000000102c5fca0 chromedriver + 457888
6   chromedriver                        0x0000000102c9f954 chromedriver + 719188
7   chromedriver                        0x0000000102c9f120 chromedriver + 717088
8   chromedriver                        0x0000000102c6a178 chromedriver + 500088
9   chromedriver  